* 本文件用来记录看书学习过程中的一些心得体会。
***
# 第一章 引言
## 1.机器学习的必要组成部分
* 训练使用的数据
* 用来转换数据的模型
* 量化模型有效性的目标函数（通常称为损失函数，最常见的是平方误差）
## 2.机器学习几种分类
* 有监督学习：带标签的数据，主要应用可以总结为回归和分类（对应连续和离散输出）
* 无监督学习：不带标签的数据，主要是学习数据的组成方式或者分布特点，如聚类分析和PCA主成分分析
* 强化学习：相比于前两种是一种在线学习，实时反馈状态和奖励的环境是得以进行训练的关键
***
# 第二章 预备知识
## 1.torch的一些使用方法
* arrange和reshape的用法类似于numpy库的array数值

In [24]:
import torch
x = torch.arange(12)
print(x)
y = x.reshape(3, 4)
print(y)

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])
tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])


* zeros,ones,randn的用法也类似于numpy库的array数组

In [25]:
print(torch.zeros((2, 3, 4)))
print(torch.ones([2, 3, 4]))
print(torch.randn(3, 4))
print(torch.tensor([[2, 1, 4, 3], [1, 2, 3, 4], [4, 3, 2, 1]]))

tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])
tensor([[[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]])
tensor([[-0.5624, -0.1966, -0.0163, -0.5957],
        [ 0.7079,  0.1652,  0.1715, -1.4762],
        [-0.6988,  2.4624,  1.6403, -0.3936]])
tensor([[2, 1, 4, 3],
        [1, 2, 3, 4],
        [4, 3, 2, 1]])


* cat可以用作连接

In [26]:
X = torch.arange(12, dtype=torch.float32).reshape((3,4))
Y = torch.tensor([[2.0, 1, 4, 3], [1, 2, 3, 4], [4, 3, 2, 1]])
print(X)
print(Y)
torch.cat((X, Y), dim=0), torch.cat((X, Y), dim=1)

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.]])
tensor([[2., 1., 4., 3.],
        [1., 2., 3., 4.],
        [4., 3., 2., 1.]])


(tensor([[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.],
         [ 2.,  1.,  4.,  3.],
         [ 1.,  2.,  3.,  4.],
         [ 4.,  3.,  2.,  1.]]),
 tensor([[ 0.,  1.,  2.,  3.,  2.,  1.,  4.,  3.],
         [ 4.,  5.,  6.,  7.,  1.,  2.,  3.,  4.],
         [ 8.,  9., 10., 11.,  4.,  3.,  2.,  1.]]))

* 索引和切片机制类似于numpy，不做赘述
* torch向量具有广播机制

In [27]:
a = torch.arange(3).reshape((3, 1))
b = torch.arange(2).reshape((1, 2))
a + b

tensor([[0, 1],
        [1, 2],
        [2, 3]])

***
## 2.自动微分

In [28]:
import torch
x = torch.arange(4.0)

x.requires_grad_(True)
print(x.grad)

y =  torch.dot(x, x)
y.backward()
print(x.grad)

None
tensor([0., 2., 4., 6.])


y.backward()的实质是将函数值计算的梯度反压回变量中，因此x.grad得以赋值
* 注意backward只能针对标量进行反向传播，这里易于理解，以上面为例,$y = x_1^2 + x_2^2 + x_3^2 + x_4^2$易于关于y求导，而$y = [x_1^2,x_2^2,x_3^2,x_4^2]$不是传统意义上的函数
解决方法如下：使用sum()将y变为标量即可

In [29]:
# 对非标量调用backward需要传入一个gradient参数，该参数指定微分函数关于self的梯度。
# 本例只想求偏导数的和，所以传递一个1的梯度是合适的
x.grad.zero_()
y = x * x
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

* 使用detatch分离变量，在复合运算时起作用

In [30]:
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x

z.sum().backward()
print(x.grad)
x.grad == u

tensor([0., 1., 4., 9.])


tensor([True, True, True, True])

* 自动微分功能强大，可以通过python程序控制流
***
## 3.矢量化加速
使用torch线性代数库做计算大大减小了开支

In [31]:
import torch
import time
n = 10000
a = torch.ones((n))
b = torch.ones([n])

class Timer:  #@save
    """记录多次运行时间"""
    def __init__(self):
        self.times = []
        self.start()

    def start(self):
        """启动计时器"""
        self.tik = time.time()

    def stop(self):
        """停止计时器并将时间记录在列表中"""
        self.times.append(time.time() - self.tik)
        return self.times[-1]

    def avg(self):
        """返回平均时间"""
        return sum(self.times) / len(self.times)

    def sum(self):
        """返回时间总和"""
        return sum(self.times)

    def cumsum(self):
        """返回累计时间"""
        return np.array(self.times).cumsum().tolist()

使用for循环的：

In [32]:
c = torch.zeros(n)
timer = Timer()
for i in range(n):
    c[i] = a[i] + b[i]
f'{timer.stop():.5f} sec'

'0.05137 sec'

使用线性代数库的：

In [33]:
timer.start()
d = a + b
f'{timer.stop():.5f} sec'

'0.00000 sec'